In [1]:
from sklearn import datasets
from scipy.optimize import minimize
import numpy as np

def load_dataset():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    
    return X, y

def train_test_split(X, y):
    idx = np.arange(len(X))
    train_size = int(len(X) * 2/3)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    return X_train, X_test, y_train, y_test

In [2]:
def init_weights(num_in, num_out):
  W = np.zeros((1 + num_in, num_out))
  sd = np.sqrt(6.0 / (num_in + num_out))
  for i in range(num_in):
    for j in range(num_out):
      x = np.float32(np.random.uniform(-sd, sd))
      W[i,j] = x
  return W


In [3]:
def sigmoid(x):
  res = 1/ (1 + np.exp(-x))
  return res

def tanh(x):
  res = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
  return res

def sigmoid_gradient(x):
  outcome = sigmoid(x)
  grad = outcome * (1 - outcome)
  return grad

def tanh_gradient(x):
  tanh_val = tanh(x)
  grad = 1 - tanh_val **2
  return grad

In [6]:
y_train.size

100

In [27]:
def forward(W, X):
    '''
    :param W: weights (including biases) of the neural network. It is a list containing both W_hidden and W_output.
    :param X: input. Already added one additional column of all "1"s.
    '''

    # Shape of W_hidden: [num_feature+1, num_hidden]
    # Shape pf W_output: [num_hidden+1, num_output]
    W1 = np.reshape(initial_W[:50],(5,10))
    W2 = np.reshape(initial_W[50:],(11,3))

    z_hidden = X.dot(W1)
    print("z1,", z_hidden.shape)
    a_hidden = tanh(z_hidden)
    print("a1",a_hidden.shape)

    a_hidden = np.concatenate([np.ones((100, 1)), a_hidden], axis=1)
    print("a1new", a_hidden.shape)
    z_output = a_hidden.dot(W2)
    print("z2",z_output.shape)

    a_output = sigmoid(z_output)
    print("a2", a_output.shape)

    #100*5 >> (5 * 10)() >> 10*3
#ip >> (100*5)
# w1 >> (5,10)
#a1 >> (100*5)
# z_h >> (100*10)
# w2 >> (11,3)
# a2 >> (100 * 11)
# z_o >> (100*3)


    return {'z_hidden': z_hidden, 'a_hidden': a_hidden,
            'z_output': z_output, 'a_output': a_output}


# Load data
#X, y = load_dataset()
#X_train, X_test, y_train, y_test = train_test_split(X, y)

# Initialize weights
#initial_W_hidden = init_weights(NUM_FEATURE, NUM_HIDDEN_UNIT)
#initial_W_output = init_weights(NUM_HIDDEN_UNIT, NUM_OUTPUT_UNIT)
#initial_W = np.concatenate([initial_W_hidden.ravel(), initial_W_output.ravel()], axis=0)

#X_input = np.concatenate([np.ones((y_train.size, 1)), X_train], axis=1)
#d = forward(initial_W, X_input)

In [20]:
#W1 = np.reshape(initial_W[:50],(5,10)).T
#X_input = np.concatenate([np.ones((y_train.size, 1)), X_train], axis=1)



In [39]:
def loss_funtion(W, X, y, num_feature, num_hidden, num_output, L2_lambda):
    '''
    :param W: a 1D array containing all weights and biases.
    :param X: input
    :param y: labels of X
    :param num_feature: number of features in X
    :param num_hidden: number of hidden units
    :param num_output: number of output units
    :param L2_lambda: the coefficient of regularization term
    '''

    m = y.size

    # Reshape W back into the parameters W_hidden and W_output
    W_hidden = np.reshape(W[:num_hidden * (num_feature + 1)],
                          ((num_feature + 1), num_hidden))

    W_output = np.reshape(W[(num_hidden * (num_feature + 1)):],
                          ((num_hidden + 1), num_output))

    # Initialize grads
    W_hidden_grad = np.zeros(W_hidden.shape)
    W_output_grad = np.zeros(W_output.shape)

    # Add one column of "1"s to X
    X_input = np.concatenate([np.ones((m, 1)), X], axis=1)

    #one hot encoding y
    num_classes = 3
    y = np.squeeze(np.eye(num_classes)[y.reshape(-1)])
    param_dict  = forward(W, X_input)
    z_hidden = param_dict['z_hidden']
    a_hidden = param_dict['a_hidden']
    z_output = param_dict['z_output']
    a_output = param_dict['a_output']


    #cross entropy with l2 regularisatn
    logprobs = np.multiply(y, np.log(a_output)) + np.multiply((1 - y), np.log(1 - a_output))
    cross_entropy_cost = (-1.0/m) * np.sum(logprobs)

    
    L2_regularization_cost = (np.sum(np.square(W_hidden)) + np.sum(np.square(W_output)))*(L2_lambda/(2*m))

    cost = cross_entropy_cost + L2_regularization_cost

    #Compute the gradient for W_hidden and W_output (including both weights and biases) 
    W2 = W_output
    W1 = W_hidden

    dZ2 = a_output - y
    dW2 = (1 / m) * np.dot(dZ2.T, a_hidden)
    dZ1 = np.multiply(np.dot(W2, dZ2.T), 1 - np.power(a_hidden.T, 2))
    dW1 = (1 / m) * np.dot(dZ1, X_input)

    #grad_hidden = (1.0/m) * np.matmul(tanh_gradient(X), np.transpose(a_hidden)) + (L2_lambda/m)*W_hidden_grad

    #grad_op = (1.0/m) * np.matmul(sigmoid_gradient(X), np.transpose(a_output)) + (L2_lambda/m)*W_output_grad


    ##########################################################################################
    # Full Mark: 3                                                                           #
    # TODO:                                                                                  #
    # 1. Transform y to one-hot encoding. Implement binary cross-entropy loss function       #
    # (Hint: Use the forward function to get necessary outputs from the model)               #
    #                                                                                        #
    # 2. Add L2 regularization to all weights in loss                                        #
    # (Note that we will not add regularization to bias)                                     #
    #                                                                                        #
    # 3. Compute the gradient for W_hidden and W_output (including both weights and biases)  #
    # (Hint: use chain rule, and don't forget to add the gradient of regularization term)    #
    ##########################################################################################
    
    L = cost
    W_hidden_grad = dW1
    W_output_grad = dW2
    
    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    grads = np.concatenate([W_hidden_grad.ravel(), W_output_grad.ravel()])

    return L, grads




In [45]:
def optimize(initial_W, X, y, num_epoch, num_feature, num_hidden, num_output, L2_lambda):
    '''
    :param initial_W: initial weights as a 1D array.
    :param X: input
    :param y: labels of X
    :param num_epoch: number of iterations
    :param num_feature: number of features in X
    :param num_hidden: number of hidden units
    :param num_output: number of output units
    :param L2_lambda: the coefficient of regularization term
    '''

    options = {'maxiter': num_epoch}
    
    ###########################################################################################
    # Full Mark: 1                                                                            #
    # TODO:                                                                                   #
    # Optimize weights                                                                        #
    # (Hint: use scipy.optimize.minimize to automatically do the iteration.)                  #
    # ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) #
    # For some optimizers, you need to set 'jac' as True.                                     #
    # You may need to use lambda to create a function with one parameter to wrap the          #
    # loss_funtion you have implemented above.                                                #
    #                                                                                         #
    # Note that scipy.optimize.minimize only accepts a 1D weight array as initial weights,    #
    # and the output optimized weights will also be a 1D array.                               #
    # That is why we unroll the initial weights into a single long vector.                    #
    ###########################################################################################
  l,grads = lambda e: loss_funtion(handler.handle, e)

     = loss_funtion(initial_W, X, y,  num_feature, num_hidden, num_output, L2_lambda)



    W_final =  minimize(,initial_W,method='BFGS')


    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    # Obtain W_hidden and W_output back from W_final
    W_hidden = np.reshape(W_final[:num_hidden * (num_feature + 1)],
                          ((num_feature + 1), num_hidden))
    W_output = np.reshape(W_final[(num_hidden * (num_feature + 1)):],
                          ((num_hidden + 1), num_output))

    return [W_hidden, W_output]


# Define parameters
NUM_FEATURE = 4
NUM_HIDDEN_UNIT = 10
NUM_OUTPUT_UNIT = 3
NUM_EPOCH = 100
L2_lambda = 1

# Load data
X, y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Initialize weights
initial_W_hidden = init_weights(NUM_FEATURE, NUM_HIDDEN_UNIT)
initial_W_output = init_weights(NUM_HIDDEN_UNIT, NUM_OUTPUT_UNIT)
initial_W = np.concatenate([initial_W_hidden.ravel(), initial_W_output.ravel()], axis=0)

# Neural network learning
W = optimize(initial_W, X_train, y_train, NUM_EPOCH, NUM_FEATURE, NUM_HIDDEN_UNIT, NUM_OUTPUT_UNIT, L2_lambda)


z1, (100, 10)
a1 (100, 10)
a1new (100, 11)
z2 (100, 3)
a2 (100, 3)


TypeError: ignored

In [ ]:
def predict(X_test, y_test, W):
    '''
    :param X_test: input
    :param y_test: labels of X_test
    :param W: a list containing two weights W_hidden and W_output.
    '''

    test_input = np.concatenate([np.ones((y_test.size, 1)), X_test], axis=1)
    d = forward(W, X)
    y_pred = d['a_output']
    ###################################################################################
    # Full Mark: 1                                                                    #
    # TODO:                                                                           #
    # Predict on test set and compute the accuracy.                                   #
    # (Hint: use forward function to get predicted output)                            #
    #                                                                                 #
    ###################################################################################

    acc = 

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    return acc